<h3>this project is created by YOUNESS AABIBI<h3>
<h3>the goal of this project is to detect objects in images using only opencv<h3>

<h5>task1: detect objects in images<h5>


<h5>method 1 : using cv2.canny()<h5>

In [112]:
import numpy as np
import cv2

In [113]:
# read img
img = cv2.imread('imgs/creeper.webp')
# showing img
cv2.imshow('image', img)
# wait for key to close
cv2.waitKey(0)
# destroy all windows
cv2.destroyAllWindows()

In [114]:
# convert the img to black and white such that the edges of an object is in white and rest is black
# to convert this img we will us canny fct of opencv
canny = cv2.Canny(img, 20, 200)
cv2.imshow('image2', canny)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [115]:

def detect_edgs_of_object(canny):
    """
        this fct detects the edges of an object in an img we check if the pixel is white or black then we get the max and min of x and y 
        so we can draw a rectangle around the object
        param: canny: the result of canny fct
        return: max_x,min_x,max_y,min_y: the max and min of x and y
    """
    max_x,min_x,max_y,min_y = [0,10000000,0,10000000]
    height,width = canny.shape
    print(canny.shape)
    for i in range(0, height):             
        for j in range(0, (width)):
            if canny[i,j]==255:
                if j>=max_x:
                    max_x=j
                if j<=min_x:
                    min_x=j
                if i>=max_y:
                    max_y=i
                if i<=min_y:
                    min_y=i
    return max_x,min_x,max_y,min_y

In [116]:
"""
    the problem with this methode is that we can detect just one object in the img
"""
max_x,min_x,max_y,min_y = detect_edgs_of_object(canny)
cv2.rectangle(img,(min_x,min_y), (max_x, max_y),(0,255,0),2)      
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows() 

(1290, 860)


<h5>method 2 : using cv2.canny() and cv2.findContours()<h5>

In [65]:
"""
    this methode draw a contour around the objects in the img
    to work with this methode we need to convert the img to canny img or using the threshold fct
    but i will use the canny img becaus it is more clear and also it is more accurate
    
"""
# read img
img = cv2.imread('imgs/phone.webp')
# convert to canny img 
img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

# equalize the histogram of the Y channel
img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

# convert the YUV image back to RGB format
img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
blurred = cv2.GaussianBlur(img, (15, 15), 0)
hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
blurred = cv2.GaussianBlur(hsv, (9, 9), 0)
canny = cv2.Canny(blurred, 30, 200)
# we can clear the noise in the img by using the Dilation fct in opencv
kernel = np.ones((1, 1), np.uint8)
# dilate the img
img_dilate = cv2.erode(canny, kernel, iterations=1)
# find the contours
contours, hierarchy = cv2.findContours(img_dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
# draw the contours
print(len(contours))
img_copy=img.copy()
maxx=[0,0,0]
maxy=[0,0,0]
maxw=[0,0,0]
maxh=[0,0,0]
j=0
maxarea=[0,0,0]
for i,c in enumerate(contours):
    x,y,w,h = cv2.boundingRect(c)
    for k in range(len(maxarea)):
        if w*h>maxarea[k]:
            maxarea[k]=w*h
            maxx[k]=x
            maxy[k]=y
            maxw[k]=w
            maxh[k]=h
            break
print(maxarea)
for i in range(len(maxarea)):
    cv2.rectangle(img, (maxx[i]+20,maxy[i]+20), (maxx[i]+maxw[i], maxy[i]+maxh[i]), (255, 255, 0), 2)
# show the img
cv2.imshow('img', img)
cv2.imshow('img_dilate', img_dilate)
cv2.imshow('canny', canny)
cv2.imshow('img_copy', img_copy)
cv2.waitKey(0)
cv2.destroyAllWindows()






15
[56672, 54312, 35280]


<h5>method 3: using cascade classifier<h5>


In [118]:
"""
    to use cascade classifier we need to download the xml file from the internet
    the down side of this method is that we need xml file for each object we want to detect
"""
img = cv2.imread('imgs/stop.jpg')
# read the xml file
cascade_classifier = cv2.CascadeClassifier('haarcascade/stop_data.xml')
# find data
data = cascade_classifier.detectMultiScale(img, 1.1, 5)
# draw rectangle around the object
if len(data):
    for (x, y, w, h) in data:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
# show the img
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()



<h5>method 4: using cv2.dnn method<h5>

In [3]:
"""
    this methode is the best one to detect objects in an img because it uses deep learning model to detect objects
    dnn stands for deep neural network
    so first thing we need to do is to download the pre-trained model to detect objects (you will find the model inside  folder caled model)  
    the model's size is 13 MB so if you dont want to download it you can skip this method
"""
import cv2
import numpy as np

In [55]:
img = cv2.imread('imgs/phone.webp')
cv2.imshow('Output', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [56]:
net = cv2.dnn_DetectionModel('model/frozen_inference_graph.pb', 'model/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt')

In [57]:
net.setInputSize(320 , 230)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

< cv2.dnn.Model 00000219BED93630>

In [58]:


classIds, confs, bbox = net.detect(img, confThreshold=0.5)
index = cv2.dnn.NMSBoxes(bbox, confs, 0.5, 0.4)

for i, box in enumerate( bbox):
    if i in index:
        cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)


cv2.imshow('Output', img)
cv2.waitKey(0)

cv2.destroyAllWindows()

<h3>task 2 : face detection in livecam using opencv<h3>

In [15]:
import cv2
import numpy as np


In [16]:
"""
    to detect faces using opencv we can use CascadeClassifier fct 
"""
cap = cv2.VideoCapture(0)
# read the xml file
cascade_face = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt.xml')
while True:
    # read frame
    ret, frame = cap.read()
    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # detecting the face in the frame
    found = cascade_face.detectMultiScale(frame, 1.1,5)
    # if face is found draw rectangle around the face
    if len(found):
        # draw a rectangle around the face  
        for (x,y,widht,height) in found:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (0,255,0), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

<h3> task 3: face and hand detaction <h3> 

In [21]:
"""
    to detect face and hands we will use cv2.CascadeClassifier fct but there is problem with that the hand is not detected well
"""
import cv2
import numpy as np

In [22]:
cap = cv2.VideoCapture(0)
# read the xml file
cascade_face = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt.xml')
cascade_hand = cv2.CascadeClassifier("haarcascade/1256617233-1-haarcascade_hand.xml")
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_found = cascade_face.detectMultiScale(gray, 1.1,5)
    hand_found = cascade_hand.detectMultiScale(gray, 1.1,5)
    if len(face_found):
        for (x,y,widht,height) in face_found:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (255,255,0), 2)
    if len(hand_found):
        for (x,y,widht,height) in hand_found:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (0,255,0), 2)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [68]:
"""
    to fix this problem i tried to mask the frame 
"""
#reading the input webcam
cap = cv2.VideoCapture(0)
# hand_cascade = cv2.CascadeClassifier('./haarcascade/haarcascade_hand.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt.xml')
palm_cascade = cv2.CascadeClassifier('./haarcascade/1256617233-1-haarcascade_hand.xml')
while True :
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([235,173,127],np.uint8)
    imageYCrCb = cv2.cvtColor(frame,cv2.COLOR_BGR2YCR_CB)
    skinRegionYCrCb = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
    skinYCrCb = cv2.bitwise_and(frame, frame, mask = skinRegionYCrCb)
    #Erosion
    kernel = np.ones((1,1), np.uint8)

    erode_image = cv2.erode(skinYCrCb,kernel, iterations=1)
    
    cv2.imshow('image', skinYCrCb)
    found_palm = palm_cascade.detectMultiScale(skinYCrCb, 1.1,25)
    found_face = face_cascade.detectMultiScale(frame, 1.1,5)
    
    if len(found_palm):
        for (x,y,widht,height) in found_palm:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (0,255,0), 2)
    if len(found_face):
        for (x,y,widht,height) in found_face:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (255,255,0), 2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



<h5>task 4 draw with your hand<h5>

In [73]:
import cv2
import numpy as np

In [76]:
cap = cv2.VideoCapture(0)
hand_cascade = cv2.CascadeClassifier('./haarcascade/1256617233-1-haarcascade_hand.xml')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_alt.xml')
points = []
blank = np.zeros([512,512,3],dtype=np.uint8)
blank.fill(0)
fx,fy,fw,fh = [0,0,0,0]
while True :
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([235,173,127],np.uint8)
    imageYCrCb = cv2.cvtColor(frame,cv2.COLOR_BGR2YCR_CB)
    skinRegionYCrCb = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
    skinYCrCb = cv2.bitwise_and(frame, frame, mask = skinRegionYCrCb)
    #Erosion
    kernel = np.ones((2,2), np.uint8)

    erode_image = cv2.erode(skinYCrCb,kernel, iterations=1)
    found = hand_cascade.detectMultiScale(erode_image, 1.1,25)
    found_face = face_cascade.detectMultiScale(frame, 1.1,5)
    if len(found):
        
        if len(found_face):
            fx,fy,fw,fh = found_face[0]
        for (x,y,widht,height) in found:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (0,255,0), 2)
            center = (x + widht//2, y + height//2)
            cv2.circle(frame, center, 20, (255, 0, 0), 2)
            if center[0]>= fx and center[0] <= fx+fw and center[1] >= fy and center[1] <= fy+fh:
                points=[] 
            elif len(points) <2:
                points.append(center)
            else:
                for i in range(1,len(points)):
                    cv2.line(blank, points[i-1], points[i], (0, 0, 255), 2)
                points.append(center)
    if len(found_face):
        for (x,y,widht,height) in found_face:
            cv2.rectangle(frame, (x,y), (x+widht, y+height), (255,0,0), 2)
    cv2.imshow('frame', frame)
    cv2.imshow('image', blank)
    if cv2.waitKey(1) == ord('q'):
        break
    
cap.release()


cv2.destroyAllWindows()

"""
    this fct doesnt prforme well because the hand haarcascaed is bad 
"""

'\n    this fct doesnt prforme well because the hand haarcascaed is bad \n'

In [75]:
import cv2
import numpy as np


In [76]:
model = cv2.dnn.readNet("model/cross-hands.weights","model/cross-hands.cfg")
model

< cv2.dnn.Net 000001BCFFC82F90>

In [77]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    output_layers=[]
    for i in net.getUnconnectedOutLayers():
        output_layers.append(layer_names[i-1])

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    color = (255,255)

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

In [78]:
"""
this model is so heavy so if you dont want to download it you can skip this method
this method just detect hands
"""

cap = cv2.VideoCapture(0)
while True :
    ret,img = cap.read()
    img = cv2.flip(img,1)
    Height, Width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), (0,0,0), True, crop=False)


    model.setInput(blob)
    
    # run inference through the network
    # and gather predictions from output layers
    outs = model.forward(get_output_layers(model))
    
    # initialization
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    # for each detetion from each output layer 
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    img_copy = img.copy()
    # go through the detections remaining
    # after nms and draw bounding box
    for i in indices:
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_bounding_box(img_copy, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    cv2.imshow("object detection", img_copy)
    cv2.imshow("original", img)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

"""
    to detect hands we will use mediapipe library
"""

'\n    to detect hands we will use mediapipe library\n'